In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import re

# Data Wrangling

In [ ]:
data=pd.read_csv('/kaggle/input/datacamp-pet-sales/pet_sales.csv')

In [ ]:
data.head(7)

In [ ]:
print('shap of data is: ',data.shape)


In [ ]:
data.describe(include='all')

In [ ]:
data.info()

In [ ]:
print( 'total number of products:',len(data['product_id'].unique()))

In [ ]:
data.isnull().sum()

In [ ]:
data.dtypes

# Data Cleaning
* changed sales column to integer
* Also changed product_Id column to string

In [ ]:
data['sales']=  data['sales'].apply( lambda sale: re.sub("[^0-9]", "", sale)   )
data['sales']=  data['sales'].apply( lambda sale: int(sale)   )
data['product_id']=  data['product_id'].apply( lambda pid: str(pid)   )
data.head()

I need only 3 categories; dog,fish, and bird and i am going to filter the dataset for them

In [ ]:
data=data[(data['pet_type']=='dog')| (data['pet_type']=='cat')| (data['pet_type']=='fish')]
data.shape

sort the data into two: products with more than one purchase and products with only one purchase

In [ ]:
data_r_1=data[data['re_buy']==1]
data_r_0=data[data['re_buy']==0]

In [ ]:
data['re_buy'].unique()

In [ ]:
data_r_1['pet_type'].unique()

# EDA

Get the total number of products which have a resale  and those which do not 

In [ ]:
unique_vals= data['re_buy'].unique()
unique_vals_count=[]
unique_vals_names=[]
for x in unique_vals:
    data_x=data[data['re_buy']==x]
    unique_vals_count.append(data_x.shape[0])
    if x==1:
        unique_vals_names.append('Product with at least 2 re_buys:'+ str(data_x.shape[0]))
    else:
        unique_vals_names.append('Product with exactly 1 purchase: '+ str(data_x.shape[0]))
print(unique_vals_count)

In [ ]:
#show plot 
#define Seaborn color palette to use
colors = sns.color_palette('pastel')[0:7]
plt.pie(unique_vals_count,labels=unique_vals_names,autopct='%1.1f%%',startangle=90,colors=colors)
#plt.legend()
plt.show()

I will do a bar plot showing sales of products purchased with more than once

In [ ]:

plt.figure(figsize=(28,25))
plt.plot(data_r_1['product_id'],data_r_1['sales'])
plt.show()

In [ ]:
data_r_1=data_r_1.sort_values(by='sales',ascending=False)

In [ ]:
data_r_0=data_r_0.sort_values(by='sales',ascending=False)

In [ ]:
new_df=pd.concat([data_r_1.head(),data_r_0.head()])
new_df.sort_values(by='sales',ascending=False,inplace=True)

I want to visualize the first 10 product with 5 highest sales taken from both data_r_0 and data_r_1

In [ ]:
'''
plt.figure(figsize=(20,20))
plt.plot(new_df['pet_type'],new_df['sales'])

plt.show()
'''

Checking the total sales of products categorized by whether or not they had rebuys.

In [ ]:
print('total sales for products with no more than one purchase: $'+ str(data_r_0['sales'].sum()))
print('total sales for products with  more than one purchase: $'+ str(data_r_1['sales'].sum()))

To answer the question: What products are more likely to be purchased again for different types of pets?

In [ ]:
group_pet=data_r_1.groupby('pet_type')


In [ ]:
group_pet.get_group('cat').head()

In [ ]:
group_pet.get_group('fish').shape

In [ ]:
group_pet.get_group('dog').head()

In [ ]:
group_pet.get_group('dog').shape

In [ ]:
cat=data_r_1.groupby('product_category')
food_cat=cat.get_group('Food')


In [ ]:
plt.figure(figsize=(30,25))
plt.bar(food_cat['product_id'],food_cat['sales'],label='Products and their sales for food category')
plt.xlabel('Product Ids',fontsize=24,color='r')
plt.ylabel('Total sales in the past year',fontsize=24)
plt.xticks(rotation='vertical',fontsize=20)
plt.yticks(fontsize=20)
plt.show()

View products likely to be purchased again for pet type dog

In [ ]:
type_dog=group_pet.get_group('dog')
plt.figure(figsize=(20,70))
plt.barh(type_dog['product_id'],type_dog['sales'],label='Products and their sales for dog pet',height=0.9)
plt.xlabel('Product Ids',fontsize=24,color='r')
plt.xlabel('Total sales in the past year',fontsize=24,color='r')
plt.yticks(fontsize=20) #rotation='vertical',
plt.yticks(fontsize=20)
plt.savefig('dog-products.png')
plt.show()